# Proyecto PixelSounds

PixelSounds es el resultado al que hemos llegado entre todos los participantes de este grupo al querer responder a nuestra manera la misma cuestión: si es posible  

Algo importante sobre este proyecto creo que también es que no hay un resultado definitivo de nada, todo ha sido probar cosas nuevas, añadir a ver que pasa si hacemos algo nuevo, ver la evolución de una idea y cómo la curiosidad puede alimentar ideas para desarrollar conocimientos